# GenX secondary analysis on GCP -  GATK with new config

bring in all the code repos needed to config the pipeline

In [ ]:
%bash
git clone https://github.com/googlegenomics/pipelines-api-examples.git  #to run it on GCP
git clone https://github.com/broadinstitute/wdl.git                     #the actual wdl scripts running the algos  
git clone https://github.com/openwdl/wdl.git open_wdl                   # use a yaml from G (skip if you write your own)  

verify 

In [6]:
%%bash
ls

GATK_new.ipynb
gen_venv
open_wdl
pipelines-api-examples
wdl


In [3]:
pwd

'/Users/sergiovillani/GenX'

take a look at a sample input containing all the data to be analyzed

In [7]:
%%bash
cat wdl/scripts/broad_pipelines/germline-short-variant-discovery/gvcf-generation-per-sample/1.0.0/GOTC_PairedEndSingleSampleWf.inputs.json

{
  "##_COMMENT1": "SAMPLE-SPECIFIC - MUST BE ADAPTED TO YOUR DATA !!! (fingerprint file is optional -- comment out if unavailable)",
  "PairedEndSingleSampleWorkflow.sample_name": "NA12878",
  "PairedEndSingleSampleWorkflow.base_file_name": "NA12878",
  "PairedEndSingleSampleWorkflow.flowcell_unmapped_bams": [
    "gs://genomics-public-data/test-data/dna/wgs/hiseq2500/NA12878/H06HDADXX130110.1.ATCACGAT.20k_reads.bam",
    "gs://genomics-public-data/test-data/dna/wgs/hiseq2500/NA12878/H06HDADXX130110.2.ATCACGAT.20k_reads.bam",
    "gs://genomics-public-data/test-data/dna/wgs/hiseq2500/NA12878/H06JUADXX130110.1.ATCACGAT.20k_reads.bam"
  ],
  "PairedEndSingleSampleWorkflow.final_gvcf_name": "NA12878.g.vcf.gz",
  "PairedEndSingleSampleWorkflow.fingerprint_genotypes_file": "gs://dsde-data-na12878-public/NA12878.hg38.reference.fingerprint.vcf",

  "##_COMMENT2": "INTERVALS", 
  "PairedEndSingleSampleWorkflow.wgs_calling_interval_list": "gs://broad-references/hg38/v0/wgs_calling_regions.hg38

take a look at the config file

In [9]:
%%bash
cat open_wdl/runners/cromwell_on_google/wdl_runner/wdl_pipeline.yaml \

name: WDL Runner
description: Run a workflow defined by a WDL file

inputParameters:
- name: WDL
  description: Workflow definition
- name: WORKFLOW_INPUTS
  description: Workflow inputs
- name: WORKFLOW_OPTIONS
  description: Workflow options

- name: WORKSPACE
  description: Cloud Storage path for intermediate files
- name: OUTPUTS
  description: Cloud Storage path for output files

docker:
  imageName: gcr.io/broad-dsde-outreach/wdl_runner:2017_10_02

  cmd: >
    /wdl_runner/wdl_runner.sh

resources:
  minimumRamGb: 3.75


create bucket (if you don't have it)

In [10]:
%%bash
gsutil ls

gs://sv-ml-sandbox/
gs://sv-tf-model-1/
gs://sv-vdv-output1/


look at the output results

configure the pipeline and kick it off

In [5]:
%%bash

gcloud alpha genomics pipelines run \
  --pipeline-file /content/datalab/docs/wdl/runners/cromwell_on_google/wdl_runner/wdl_pipeline.yaml \
  --zones us-central1-f \
  --memory 5 \
  --logging gs://sv-ml-sandbox/logging \
  --inputs-from-file WDL=/content/datalab/docs/intro/wdl/scripts/broad_pipelines/germline-short-variant-discovery/gvcf-generation-per-sample/0.2.0/PublicPairedSingleSampleWf_170412.wdl \
  --inputs-from-file WORKFLOW_INPUTS=/content/datalab/docs/intro/wdl/scripts/broad_pipelines/germline-short-variant-discovery/gvcf-generation-per-sample/0.2.0/PublicPairedSingleSampleWf_170412.inputs.json \
  --inputs-from-file WORKFLOW_OPTIONS=/content/datalab/docs/intro/wdl/scripts/broad_pipelines/germline-short-variant-discovery/gvcf-generation-per-sample/0.2.0/PublicPairedSingleSampleWf_170412.options.json\
  --inputs WORKSPACE=gs://sv-ml-sandbox/workspace \
  --inputs OUTPUTS=gs://sv-ml-sandbox/outputs

README.md


verify status of pipeline

In [11]:
%%bash 
gcloud alpha genomics operations describe EIPhlO6LLBjz6K6O9PS7-gMg4r3yw4IMKg9wcm9kdWN0aW9uUXVldWU --format='yaml(done, error, metadata.events)'

done: true
metadata:
  events:
  - description: start
    startTime: '2018-01-03T20:45:28.418045346Z'
  - description: pulling-image
    startTime: '2018-01-03T20:45:28.418091802Z'
  - description: localizing-files
    startTime: '2018-01-03T20:46:02.330034206Z'
  - description: running-docker
    startTime: '2018-01-03T20:46:02.330074057Z'
  - description: delocalizing-files
    startTime: '2018-01-04T00:03:35.482525370Z'
  - description: ok
    startTime: '2018-01-04T00:03:36.486153745Z'


verify pipeline stages on the pantheon  and in the bucket

In [12]:
%%bash
gsutil ls gs://sv-ml-sandbox

gs://sv-ml-sandbox/logging/
gs://sv-ml-sandbox/outputs/
gs://sv-ml-sandbox/workspace/


look at the output results

In [13]:
%%bash
gsutil ls gs://sv-ml-sandbox/outputs

gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.base_distribution_by_cycle.pdf
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.base_distribution_by_cycle_metrics
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.insert_size_histogram.pdf
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.insert_size_metrics
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_by_cycle.pdf
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_by_cycle_metrics
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_distribution.pdf
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.readgroup.quality_distribution_metrics
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.unmapped.quality_yield_metrics
gs://sv-ml-sandbox/outputs/H06HDADXX130110.1.ATCACGAT.20k_reads.validation_report
gs://sv-ml-sandbox/